In [23]:
from langchain.evaluation.qa import QAGenerateChain
from langchain.evaluation.qa import QAEvalChain

In [1]:
import os
import textwrap
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI, AzureChatOpenAI

llm = AzureChatOpenAI(
    max_tokens=16000,
    temperature=0.0,
    deployment_name="gpt-4o-b",
    model_name="gpt-4o",
    # deployment_name="gpt-35-turbo",
    # model_name="gpt-3.5-turbo",
)
print(f'Using Azure endpoint: {llm.azure_endpoint}')

# Test if LLM responds OK
prompt = "What is your training data cutoff date?"
response = llm(prompt)
print(response.content)

Using Azure endpoint: https://summarilyazureopenai.openai.azure.com


/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_68248/118664839.py:18: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)


My training data cutoff date is **October 2023**. This means I was trained on information available up until that time and do not have knowledge of events or developments that occurred after that date. Let me know how I can assist you!


### QA Chain

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()

In [4]:
from langchain.embeddings import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    deployment="text-embedding-3-small",
    chunk_size=1024)
embed = embeddings.embed_query("Hi my name is Harrison")
len(embed)

/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_68248/3486695070.py:2: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(


1536

In [5]:
docs = docs[:500]  # Limit to 500 for testing
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

/Users/mehdiazad/Documents/Projects/mini_projects/miniproj/lib/python3.13/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
retriever = db.as_retriever()

In [7]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Test data points: Manual generation

In [8]:
print("\n".join(textwrap.wrap(docs[10].page_content)))

: 10 name: Cozy Comfort Pullover Set, Stripe description: Perfect for
lounging, this striped knit set lives up to its name. We used
ultrasoft fabric and an easy design that's as comfortable at bedtime
as it is when we have to make a quick run out.  Size & Fit - Pants are
Favorite Fit: Sits lower on the waist. - Relaxed Fit: Our most
generous fit sits farthest from the body.  Fabric & Care - In the
softest blend of 63% polyester, 35% rayon and 2% spandex.  Additional
Features - Relaxed fit top with raglan sleeves and rounded hem. -
Pull-on pants have a wide elastic waistband and drawstring, side
pockets and a modern slim leg.  Imported.


In [9]:
print("\n".join(textwrap.wrap(docs[11].page_content)))

: 11 name: Ultra-Lofty 850 Stretch Down Hooded Jacket description:
This technical stretch down jacket from our DownTek collection is sure
to keep you warm and comfortable with its full-stretch construction
providing exceptional range of motion. With a slightly fitted style
that falls at the hip and best with a midweight layer, this jacket is
suitable for light activity up to 20° and moderate activity up to
-30°. The soft and durable 100% polyester shell offers complete
windproof protection and is insulated with warm, lofty goose down.
Other features include welded baffles for a no-stitch construction and
excellent stretch, an adjustable hood, an interior media port and mesh
stash pocket and a hem drawcord. Machine wash and dry. Imported.


In [10]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### Test data points: LLM assisted generation

In [11]:
from langchain.evaluation.qa import QAGenerateChain

In [12]:
example_gen_chain = QAGenerateChain.from_llm(llm=llm)

In [13]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in docs[:5]]
)

/Users/mehdiazad/Documents/Projects/mini_projects/miniproj/lib/python3.13/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [14]:
new_examples[:2]

[{'qa_pairs': {'query': "What features contribute to the Women's Campside Oxfords' comfort and support, and what specific design elements are included in the shoe's construction?",
   'answer': "The Women's Campside Oxfords are designed for comfort and support with features such as a soft canvas material for a broken-in feel, a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, and a moderate arch contour. Additional design elements include an EVA foam midsole for cushioning and support, a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern, and a vintage hunt, fish, and camping motif on the innersole."}},
 {'qa_pairs': {'query': 'What materials are used in the construction of the Recycled Waterhog Dog Mat, and what percentage of the fabric is made from recycled materials?  ',
   'answer': 'The Recycled Waterhog Dog Mat is constructed with 24 oz. polyester fabric made from 94% recycled materials and features a rubber backing.'}}]

### Combine examples

In [15]:
new_examples_processed=[]
for example in new_examples:
    new_examples_processed.append(example["qa_pairs"])        

In [16]:
examples += new_examples_processed

In [17]:
examples[:3]

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What features contribute to the Women's Campside Oxfords' comfort and support, and what specific design elements are included in the shoe's construction?",
  'answer': "The Women's Campside Oxfords are designed for comfort and support with features such as a soft canvas material for a broken-in feel, a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, and a moderate arch contour. Additional design elements include an EVA foam midsole for cushioning and support, a chain-tread-inspired molded rubber outsole with a modified chain-tread pattern, and a vintage hunt, fish, and camping motif on the innersole."}]

In [18]:
qa.run(examples[0]["query"])

/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_68248/1223946598.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])




> Entering new RetrievalQA chain...

> Finished chain.


'Yes, the Cozy Comfort Pullover Set has side pockets on the pull-on pants.'

### Manual evaluation

In [19]:
import langchain
langchain.debug = True

In [27]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

'Yes, the Cozy Comfort Pullover Set has side pockets on the pull-on pants.'

### LLM assisted evaluation

In [20]:
langchain.debug = False

In [21]:
predictions = qa.apply(examples)

/var/folders/py/lx68lx7s6sbcr2m9kwtspqvh0000gn/T/ipykernel_68248/1205324748.py:1: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  predictions = qa.apply(examples)




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [22]:
from langchain.evaluation.qa import QAEvalChain

In [24]:
eval_chain = QAEvalChain.from_llm(llm)

In [26]:
len(examples)

7

In [28]:
graded_outputs = eval_chain.evaluate(examples[:3], predictions[:3])

In [30]:
graded_outputs

[{'results': 'CORRECT'}, {'results': 'CORRECT'}, {'results': 'CORRECT'}]

In [32]:
for i, eg in enumerate(examples[:3]):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set pants have side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What features contribute to the Women's Campside Oxfords' comfort and support, and what specific design elements are included in the shoe's construction?
Real Answer: The Women's Campside Oxfords are designed for comfort and support with features such as a soft canvas material for a broken-in feel, a comfortable EVA innersole with Cleansport NXT® antimicrobial odor control, and a moderate arch contour. Additional design elements include an EVA foam midsole for cushioning and support, a chain-tread-inspired molded